## 1. Data Collection
#### 1.1 Get the list of books
The starting point of the Homework 3 is to collected the data on which we are going to work on.
What we want is to collect the url associated to each book in the list provided by this [link](https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1). \
We save all the links in the list.txt file; each line corresponds to a book's url.

In [1]:
from bs4 import BeautifulSoup
import requests
import time
import os
import re
import csv
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from field_functions import * #our script .py 
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from collections import Counter

In [ ]:
filename="page1.html"
prefix="https://www.goodreads.com"

With the following loop we extract the 100 links from each of the 300 pages:

In [ ]:
for i in range(1,301):
    filename="page"+str(i)+".html"
    directory= "G:\ADM3\\" +str(i)+"\\"
    prefix="https://www.goodreads.com"

    f=open(directory+filename,encoding="utf-8")
    lines=f.readlines()
    
    links=[]
    urls=[]
    for line in lines:
        if 'class="bookTitle"' in line:
            links.append(line)
    for link in links:
        link=prefix+re.findall(r'(?<=href=")(.*?)(?=")',link)[0] 
        urls.append(link)
    
    for url in urls:
        with open(directory+"links.txt", "a") as text_file:
            text_file.write(url+"\n")

#### 1.2 Crawl books
After collecting the urls, we download the html corresponding to each of them. We save each page into an html file in order to make the page static and also avoid problem during the process of data colletion.\
With the following loop we download the 300 pages.

In [ ]:
for i in range(1,301):
    link="https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+str(i)
    cnt = requests.get(link)
    soup = BeautifulSoup(cnt.content, features="lxml")
    name="G:\ADM3\\"+str(i)+"\\"+"page"+str(i)+".html"
    print(name)
    f = open(name, "w",encoding="utf-8")
    f.write(soup.prettify())
    f.close()
    print("Pagina "+str(i)+" finita")

Then with this third loop we download the book pages. In the for loop we go thought the 300 pages and with the second one we get from each page the all the links (100).

In [ ]:
s=requests.Session()
for i in range(1,301):
    file= "G:\ADM3\\" +str(i)+"\\"+"links.txt"
    f=open(file,encoding="utf-8")
    lines=f.readlines()
    j=1
    for url in lines:
        cnt = requests.get(url)
        soup = BeautifulSoup(cnt.content, features="html.parser")
        name="G:\ADM3\\"+str(i)+"\\"+str(j)+".html"
        f = open(name, "w",encoding="utf-8")
        f.write(soup.prettify())
        f.close()
        j+=1
    print("Pagina "+str(i)+" finita")

Actually we had run this loop more than one time on the files with size of less than 100000 bytes. Most of them were downloaded correctly this time, but some, especially in the last pages (150-300), were actually "bad pages" on 
the website. \
In the next step, during parsing, we decided to ignore pages with size of less than 100000 bytes.

### 1.3 Parse downloaded pages
Once we have the html documents containing all the books in the 300 pages, we want extract specific information from each html. \
Here the list of what we are looking:
- Title 
- Series 
- Authors
- Ratings
- Number of givent ratings 
- Number of reviews 
- The entire plot 
- Number of pages 
- Published 
- Characters
- Setting
- Url

For this purpose we define a class, one for each point in the list, and in every class there are 3 methods:
* **name()** : it returns the name of each item (bookTitle,bookSeries,ecc...)
* **parse()** : using the method soup.find() it returns the information we are looking for that are contained in the html of the web page concerning the book
* **score()** : it computes the value of the score (we use this method in point 3 of the homework) 

We decide to build this structure in case one wants to implement new fields in the future.

After collecting all the information in which we are interested, now it's time to save them properly. \
We create a file 'article_index.tsv' for each book in which the first row is the **header** and from the second one there are the information in order splitted by a tab. In this way: \
bookTitle \t bookSeries \t bookAuthors \t ratingValue \t ratingCount \t reviewCount \t Plot \t NumberOfPages \t Publishing_Date \t Characters \t Setting \t Url \
The Hunger Games \t The Hunger Games \t   ...  \t District 12, Panem, Capitol, Panem, Panem (United States) \


If one information is missing, instead of the dots there is an empty space.

In [ ]:
functions=[bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot,NumberOfPages,Publishing_Date,\
           Characters,Setting,Url]

header = ""
for fun in functions[:-1]:
    header += fun.name()+"\t"
header +=functions[-1].name()
count = 1 #for the articles

for i in range(1,301):
    for j in range(1,101):
        name="C:\\Users\\Stefania\\ADM_HW3\\ADM3\\"+str(i)+"\\"+str(j)+".html"
        try:
            with open(name,"r",encoding="utf-8") as file:
                if os.path.getsize(name)<100000: #ignoring "bad pages"
                    continue
                soup=BeautifulSoup(file,features="html.parser")
                second_line = ""
                for fun in functions[:-1]:
                    try:
                        second_line += fun.parse(soup)+"\t"
                    except:
                        print("Unspecified exception in for function ", fun.name() ," for book j" , j, "in page i " , i)
                        second_line += "" + "\t"
                try:
                    second_line += functions[-1].parse(soup)  
                except:
                    print("Unspecified exception in for function ", fun.name() ," for book j" , j, "in page i " , i) 
                    second_line += "" + "\t"
            name="C:\\Users\\Stefania\\ADM_HW3\\articles\\articles_"+str(count)+".tsv"
            with open(name,"w",encoding="utf-8") as file:
                file.write(header)
                file.write("\n")
                file.write(second_line)
            count +=1
        except FileNotFoundError:
            continue

In the *best book ever list* there are also foreign books. Since for this analysis we are only interested in english books, here we check if the plot of each book is or not in english. If not, we remove these books from our list.

In [ ]:
count = []
for i in range(1,29226):
    with open('articles/articles_' + str(i) +'.tsv', 'r', encoding="utf-8") as file:
        temp = csv.DictReader(file, delimiter = '\t')
        for row in temp:
            if custom_detect(row['Plot']) != 'en':
                count.append(i)
                print(i)

for i in count:
    os.remove('articles/articles_' + str(i) +'.tsv')

After removed the "bad plot" we reindex all the articles.

In [ ]:
counter = 1
for i in range(1, 29226):
    try:
        os.rename('articles/articles_' + str(i) +'.tsv', 'articles/article_' + str(counter) +'.tsv')
        counter += 1
    except FileNotFoundError:
        continue

## 2. Creation of structure used in Point 2
Here we create the data structures and the files used as support for the Point 2.

### Create 'vocabulary' dictionary

In [ ]:
# - keys: all words in the all documents
# - values: index of each word from 0 to 55037
tokens_set=set()
for i in range(1,N_doc+1):
    with open('F://FILE_ADM_3//articles/article_' + str(i) +'.tsv', 'r', encoding="utf-8") as file:
        temp = csv.DictReader(file, delimiter = '\t')
        for row in temp:
            #tokenizing the plot, making everything lowercase and removing unwanted characters
            Plot_words=tokenizer.tokenize(row["Plot"].lower())
            tokens_without_sw = [stemmer.stem(word) for word in Plot_words if word not in stop_words]
            temp=set(tokens_without_sw)
            
            tokens_set=tokens_set.union(temp)

#A few steps to save it in a dictionary ordered alphabetically
tokens_list=list(tokens_set)
tokens_list.sort()
tokens_tuple=[(tokens_list[i],i) for i in range(len(tokens_list))]
tokens_dictionary=dict((x, y) for x, y in tokens_tuple)

In [ ]:
# save the pkl file
with open('vocabulary.pkl', 'wb') as handle:
    pickle.dump(vocabulary, handle)

### Create 'inverted_index' dictionary

In [ ]:
# - keys: index of each word from 0 to 55037
# - values: each book containing the unique word (index number)
inverted_index={}
for i in range(len(vocabulary)):
    inverted_index[i]=[]

for j in range(1,26544):
        with open('F://FILE_ADM_3//articles/article_' + str(j) +'.tsv', 'r', encoding="utf-8") as file:
            temp = csv.DictReader(file, delimiter = '\t')
            for row in temp:
                #tokenizing the plot, making everything lowercase and removing unwanted characters
                Plot_words=tokenizer.tokenize(row["Plot"].lower())
                tokens_without_sw = [stemmer.stem(word) for word in Plot_words if word not in stop_words]
                
                for word in tokens_without_sw:
                    inverted_index[vocabulary[word]].append(j)

for i in range(len(inverted_index)):
    inverted_index[i]=set(inverted_index[i])

In [ ]:
# save the pkl file
with open('inverted_index_1.pkl', 'wb') as handle:
    pickle.dump(inverted_index_1, handle)

### Create 'vocabulary2' dictionary

In [ ]:
# - keys: index of each word from 0 to 55037
# - values: number of times a word appear in all books
vocabulary2={}
for i in range(1,26544):
    with open('F://FILE_ADM_3//articles/article_' + str(j) +'.tsv', 'r', encoding="utf-8") as file:
    #with open('articles/article_' + str(i) +'.tsv', 'r', encoding="utf-8") as file:
        temp = csv.DictReader(file, delimiter = '\t')
        for row in temp:
            #tokenizing the plot, making everything lowercase and removing unwanted characters
            Plot_words=tokenizer.tokenize(row["Plot"].lower())
            tokens_without_sw = [stemmer.stem(word) for word in Plot_words if word not in stop_words]
            temp=set(tokens_without_sw)
            for word in temp:
                if vocabulary[word] in vocabulary2:
                    vocabulary2[vocabulary[word]]+=1
                else:
                    vocabulary2[vocabulary[word]]=1

In [ ]:
# save the pkl file
with open('vocabulary2.pkl', 'wb') as handle:
    pickle.dump(vocabulary2, handle)

### Create 'tfIdf_index' dictionary

In [ ]:
# - keys: words mapped in numbers
# - values: {book in which the word appears, score of the word with respect to the book}
tfIdf_index={}
for key,value in inverted_index2.items():
    tfIdf_index[key]=[]
    n_ij=dict(Counter(value))

    for element in n_ij:
        # issue with length_documents, we count also the duplicated words
        tf=n_ij[element]/length_documents[element]
        # issue with vocabulary2, we count also the duplicated words
        Idf=np.log(N_doc/vocabulary2[key])
        
        tfIdf_index[key].append((element,tf*Idf))

In [ ]:
# save the pkl file
with open('tfIdf_index.pkl', 'wb') as handle:
    pickle.dump(tfIdf_index, handle)

### Create 'tfIdf_index' dictionary

In [ ]:
BookTokens={}
for i in range(1,N_doc+1):
    with open('F://FILE_ADM_3//articles/article_' + str(i) +'.tsv', 'r', encoding="utf-8") as file:
        temp = csv.DictReader(file, delimiter = '\t')
        for row in temp:
            #tokenizing the plot, making everything lowercase and removing unwanted characters
            Plot_words=tokenizer.tokenize(row["Plot"].lower())
            tokens_without_sw = [stemmer.stem(word) for word in Plot_words if word not in stop_words]
            #temp=set(tokens_without_sw)
            BookTokens[i]=[]
            n_ij=dict(Counter(tokens_without_sw))
            for word in tokens_without_sw:
                tf=n_ij[word]/length_documents[i]
                Idf=np.log(N_doc/vocabulary2[vocabulary[word]])
                BookTokens[i].append((vocabulary[word],tf*Idf))
                BookTokens[i]=list(set(BookTokens[i]))
                BookTokens[i].sort()

In [ ]:
# save the pkl file
with open('BookTokens.pkl', 'wb') as handle:
    pickle.dump(BookTokens, handle)